### Регрессионный анализ в полном факторном эксперименте (ПФЭ 2³)
<hr style="border:2px solid gray">

## Задача
Исследуем влияние трёх факторов на целевую переменную $ y $ — **предел текучести стали $ \sigma_{02} $, МПа**:
- $ x_1 $: содержание углерода (0.06% – 0.10%)
- $ x_2 $: толщина материала (0.7 мм – 1.2 мм)
- $ x_3 $: угол относительно направления проката (0° – 90°)

Построим модель в виде линейной регрессии:
$$
\hat{y} = b_0 + b_1 x_1 + b_2 x_2 + b_3 x_3
$$

<hr style="border:2px solid gray">

В ходе анализа проверим:
- Однородность дисперсий опытов — **критерий Кохрена**
- Значимость коэффициентов — **критерий Стьюдента**
- Адекватность модели — **критерий Фишера**

---

### Экспериментальные данные

Проведён **Полный Факторный Эксперимент (ПФЭ 2³)** — 3 фактора на 2 уровнях, всего 8 опытов.  
Каждый опыт повторён 3 раза для оценки ошибки воспроизводимости.

Данные получены для сталей **DC01** и **DC04**:

- **DC01**: $ C = 0.10\% $
- **DC04**: $ C = 0.06\% $

---

### Матрица планирования в натуральных переменных

| № опыта | Содержание углерода, % | Толщина, мм | Угол проката, ° | $ y_1 $, МПа | $ y_2 $, МПа | $ y_3 $, МПа |
|:-------:|:----------------------:|:-----------:|:---------------:|:------------:|:------------:|:------------:|
| 1       | 0.10                   | 0.7         | 0               | 238          | 237          | 239          |
| 2       | 0.06                   | 0.7         | 0               | 160          | 161          | 159          |
| 3       | 0.10                   | 1.2         | 0               | 240          | 241          | 239          |
| 4       | 0.06                   | 1.2         | 0               | 186          | 185          | 187          |
| 5       | 0.10                   | 0.7         | 90              | 243          | 244          | 242          |
| 6       | 0.06                   | 0.7         | 90              | 164          | 163          | 165          |
| 7       | 0.10                   | 1.2         | 90              | 242          | 243          | 241          |
| 8       | 0.06                   | 1.2         | 90              | 201          | 200          | 202          |


In [1]:
import numpy as np
from itertools import product

# Настройки отображения
np.set_printoptions(precision=4, suppress=True)

## 1.  📌  Матрица планирования кодируется таким образом, что:
- низкий уровень → -1 (например, если содержание углерода равно 0.06%) 
- высокий уровень → +1 (например, если содержание углерода равно 0.1%)

 **Общий вид кодирования фактора:**
$$
x_j = \frac{X_j - X_{j0}}{\Delta_j}
$$
где:
- $ x_j $ — кодированное значение фактора (безразмерное, $ -1 $ или $ +1 $)
- $ X_j $ — натуральное значение фактора
- $ X_{j0} = \frac{X_{j\text{max}} + X_{j\text{min}}}{2} $ — **центр уровня** (середина интервала)
- $ \Delta_j = \frac{X_{j\text{max}} - X_{j\text{min}}}{2} $ — **интервал варьирования**

в нашем случае:

$$
x_1 = \frac{C - 0.08}{0.02},\quad
x_2 = \frac{h - 0.95}{0.25},\quad
x_3 = \frac{\theta - 45}{45}
$$

In [2]:
# Порядок: x1 (C), x2 (h), x3 (θ)
# +1: верхний уровень, -1: нижний уровень
X_coded = np.array([
    [+1, -1, -1],  # Опыт 1: C=0.10 (+1), h=0.7 (-1), θ=0 (-1)
    [-1, -1, -1],  # Опыт 2: C=0.06 (-1), h=0.7 (-1), θ=0 (-1)
    [+1, +1, -1],  # Опыт 3: C=0.10 (+1), h=1.2 (+1), θ=0 (-1)
    [-1, +1, -1],  # Опыт 4: C=0.06 (-1), h=1.2 (+1), θ=0 (-1)
    [+1, -1, +1],  # Опыт 5: C=0.10 (+1), h=0.7 (-1), θ=90 (+1)
    [-1, -1, +1],  # Опыт 6: C=0.06 (-1), h=0.7 (-1), θ=90 (+1)
    [+1, +1, +1],  # Опыт 7: C=0.10 (+1), h=1.2 (+1), θ=90 (+1)
    [-1, +1, +1],  # Опыт 8: C=0.06 (-1), h=1.2 (+1), θ=90 (+1)
])

print("Матрица планирования в кодированном масштабе:")
print("x1 (C)  x2 (h)  x3 (θ)")
print(X_coded)

Матрица планирования в кодированном масштабе:
x1 (C)  x2 (h)  x3 (θ)
[[ 1 -1 -1]
 [-1 -1 -1]
 [ 1  1 -1]
 [-1  1 -1]
 [ 1 -1  1]
 [-1 -1  1]
 [ 1  1  1]
 [-1  1  1]]


In [3]:
# Значения целевой переменной: каждая строка — 3 повторения одного и того же из восьми опытов
y_repeated = np.array([
    [238.5, 237.0, 239.2],  # Опыт 1: C=0.10%, h=0.7 мм, θ=0°  → DC01
    [160.9, 161.4, 158.0],  # Опыт 2: C=0.06%, h=0.7 мм, θ=0°  → DC04
    [240.2, 241.9, 239.0],  # Опыт 3: C=0.10%, h=1.2 мм, θ=0° → DC01
    [183.1, 185.4, 187.8],  # Опыт 4: C=0.06%, h=1.2 мм, θ=0° → DC04
    [241.4, 244.8, 242.3],  # Опыт 5: C=0.10%, h=0.7 мм, θ=90° → DC01
    [165.6, 163.2, 165.5],  # Опыт 6: C=0.06%, h=0.7 мм, θ=90° → DC04
    [243.1, 243.1, 241.2],  # Опыт 7: C=0.10%, h=1.2 мм, θ=90° → DC01
    [201.9, 198.3, 202.1],  # Опыт 8: C=0.06%, h=1.2 мм, θ=90° → DC04
])

print("Отклики с повторениями:")
print(y_repeated)

Отклики с повторениями:
[[238.5 237.  239.2]
 [160.9 161.4 158. ]
 [240.2 241.9 239. ]
 [183.1 185.4 187.8]
 [241.4 244.8 242.3]
 [165.6 163.2 165.5]
 [243.1 243.1 241.2]
 [201.9 198.3 202.1]]


## 2.  📌  Расчет среднего значения и построчной дисперсии

После проведения эксперимента, где **каждый опыт повторяется несколько раз** (в нашем случае — 3 раза), важно не просто записать результаты, а **охарактеризовать каждый опыт двумя показателями**:

1. **Среднее значение отклика** — чтобы понять, *что показал опыт в среднем*.
2. **Построчная дисперсия** — чтобы понять, *насколько стабильны измерения*.

### 1. Среднее значение целевой переменной

**Формула:**
$$
\bar{y}_i = \frac{1}{m} \sum_{j=1}^m y_{ij}
$$

Где:
- $ \bar{y}_i $ — среднее значение отклика для **i-го опыта**
- $ m $ — число повторений (в нашем случае $ m = 3 $)
- $ y_{ij} $ — j-е измерение в i-м опыте



### 2. Построчная дисперсия

**Формула:**
$$
S_i^2 = \frac{1}{m-1} \sum_{j=1}^m (y_{ij} - \bar{y}_i)^2
$$

Где:
- $ S_i^2 $ — дисперсия i-го опыта
- $ m-1 $ — число степеней свободы (для несмещённой оценки)
- $ y_{ij} - \bar{y}_i $ — отклонение каждого измерения от среднего


Почему именно $ m-1 $?

При расчёте построчной дисперсии мы используем **среднее, вычисленное по тем же данным**, поэтому теряется одна степень свободы.  
Деление на $ m-1 $ (а не на $ m $) даёт **несмещённую оценку** истинной дисперсии.


**Зачем нужна построчная дисперсия:**
- Показывает **разброс данных** в рамках одного опыта.
- Чем больше дисперсия — тем **менее стабилен процесс** при этих факторах.
- Используется для:
  - Проверки **однородности дисперсий** (критерий Кохрена)
  - Оценки **ошибки эксперимента**
  - Проверки **адекватности модели** (критерий Фишера)

>⚠️ Если дисперсия одного опыта сильно больше других — возможно, в нём была ошибка (например, сбой оборудования).

In [4]:
m = y_repeated.shape[1]  # число повторений = 3
n = y_repeated.shape[0]  # число опытов = 8

y_mean = np.mean(y_repeated, axis=1)  # (8,)
S2 = np.var(y_repeated, axis=1, ddof=1)  # (8,)

print("Опыт | Среднее | Дисперсия")
print("-----|---------|----------")
for i in range(n):
    print(f" {i+1:2}  |  {y_mean[i]:5.1f}  |  {S2[i]:6.3f}")

Опыт | Среднее | Дисперсия
-----|---------|----------
  1  |  238.2  |   1.263
  2  |  160.1  |   3.370
  3  |  240.4  |   2.123
  4  |  185.4  |   5.523
  5  |  242.8  |   3.103
  6  |  164.8  |   1.843
  7  |  242.5  |   1.203
  8  |  200.8  |   4.573


## 3. 📌 Проверка однородности дисперсий по критерию Кохрена

$$
G = \frac{\max(S_i^2)}{\sum_{i=1}^n S_i^2}
$$

Полученное расчетное значение критерия Кохрена необходимо сравнить с табличным при $ \alpha = 0.05 $:
| $ n $ ↓ \\ $ f $ → | 2       | 3       | 4       | 5       | 6       |
|---------------------|---------|---------|---------|---------|---------|
| **2**               | 0.998   | 0.967   | 0.906   | 0.841   | 0.781   |
| **3**               | 0.975   | 0.874   | 0.768   | 0.684   | 0.623   |
| **4**               | 0.939   | 0.798   | 0.684   | 0.600   | 0.539   |
| **5**               | 0.906   | 0.746   | 0.633   | 0.551   | 0.494   |
| **6**               | 0.877   | 0.707   | 0.599   | 0.520   | 0.465   |
| **7**               | 0.850   | 0.677   | 0.574   | 0.498   | 0.444   |
| **8**               | 0.828   | 0.653   | 0.554   | 0.479   | 0.427   |
| **9**               | 0.808   | 0.633   | 0.537   | 0.463   | 0.412   |
| **10**              | 0.791   | 0.615   | 0.522   | 0.450   | 0.401   |

- $ n $ — число опытов (или число групп дисперсий)
- $ f = m - 1 $ — число степеней свободы **одной** выборки (где $ m $ — число повторений
- $ \alpha $ - уровень значимости

$ \alpha = 0.05 $ означает, что мы допускаем 5% вероятность ошибочно заключить, что дисперсии неоднородны.

> 🔹 Пример использования приведенной выше таблицы:  
> У нас 8 опытов ($ n = 8 $), каждый повторён 3 раза → $ f = m - 1 = 2 $  
> Тогда критическое значение: $ G_{\text{крит}} = 0.828 $  
> Если расчетное $ G_{\text{набл}} < 0.828 $ → дисперсии однородны.

In [5]:
# Расчетный критерий Кохрена
G_observed = np.max(S2) / np.sum(S2)
# Табличный критерий Кохрена
G_critical = 0.828  # табличное значение

print(f"Расчетный критерий Кохрена G: {G_observed:.4f}")
print(f"Табличный критерий Кохрена G: {G_critical}")

if G_observed < G_critical:
    print("✅ Дисперсии однородны. Можно продолжать анализ.")
else:
    print("❌ Дисперсии неоднородны. Модель может быть ненадёжной.")

Расчетный критерий Кохрена G: 0.2401
Табличный критерий Кохрена G: 0.828
✅ Дисперсии однородны. Можно продолжать анализ.


## 4. 📌 Средняя дисперсия воспроизводимости (сумма построчных дисперсий)

$$
S^2_{\text{воспр}} = \frac{1}{n} \sum_{i=1}^n S_i^2
$$

понадобится нам чуть позднее

In [6]:
S2_vospr = np.mean(S2)
print(f"Средняя дисперсия воспроизводимости: {S2_vospr:.3f}")

Средняя дисперсия воспроизводимости: 2.875


## 5. 📌 Построение модели на основе метода наименьших квадратов

В матрица плана X необходимо добавить столбец единиц для последующего рассчета свободного члена уравнения регрессии (коэффициента $ b_0$):
$$
\mathbf{X} = [\mathbf{1},\ \mathbf{x}_1,\ \mathbf{x}_2,\ \mathbf{x}_3]
$$

Коэффициенты линейной регрессии расчитываются с помощью следующего матричного уравнения:
$$
\mathbf{b} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

In [7]:
# добавляем столбец с единицами в матрицу планирования в кодированном масштабе
X = np.column_stack([np.ones(n), X_coded])  # (8, 4)
# создаем вектор отклика равный вектору средних значений целевой переменной
y = y_mean

# Расчет вектора коэффициентов с помощью инструментов перемножения матриц библиотеки NumPy
XTX = X.T @ X
XTX_inv = np.linalg.inv(XTX)
b = XTX_inv @ X.T @ y

print("Коэффициенты модели:")
print(f"b0: {b[0]:.2f}")
print(f"b1: {b[1]:.2f}")
print(f"b2: {b[2]:.2f}")
print(f"b3: {b[3]:.2f}")

Коэффициенты модели:
b0: 209.37
b1: 31.60
b2: 7.89
b3: 3.34


## 6.📌 Проверка значимости коэффициентов по критерию Стьюдента


После построения модели важно понять какие коэффициенты действительно влияют на отклик, а какие — просто "шум"?

Для этого используется **критерий Стьюдента (t-критерий)**.

---

### 🔢 Как проводится проверка?

#### 1. **Стандартная ошибка коэффициента**

Показывает, **насколько точно** мы оценили коэффициент.

Для ортогонального плана ПФЭ 2³:
$$
\text{SE}(b_j) = \sqrt{\frac{S^2_{\text{воспр}}}{N}}, \quad j \ge 1
$$

Где:
- $ S^2_{\text{воспр}} $ — средняя дисперсия воспроизводимости
- $ N = 8 $ — число опытов
- $ j $ - номер коэффициента, который мы проверяем на значимость



#### 2. Расчетный **t-критерий**

$$
t_j = \frac{|b_j|}{\text{SE}(b_j)}
$$

Чем больше $ |b_j| $ по сравнению с ошибкой — тем **выше уверенность**, что коэффициент значим.


#### 3. Табличный **t-критерий**

Сравниваем $ t_j $ с табличным значением $ t_{\text{крит}} $, которое зависит от:

- $ f $ — числа степеней свободы: $ f = n \cdot (m - 1) $
- $ \alpha = 0.05 $ — уровня значимости

В нашем случае:
- $ n = 8 $ опытов
- $ m = 3 $ повторения
- $ f = 8 \cdot (3 - 1) = 16 $
- $ t_{\text{крит}}(f=16,\ \alpha=0.05) = 2.120 $

Если $ t_j > t_{\text{крит}} $, то коэффициент **статистически значим**.



### Таблица: Критические значения t-критерия Стьюдента ($ \alpha = 0.05 $)

| Число степеней свободы $ f $ | $ t_{\text{крит}} $ |
|-------------------------------|----------------------|
| 1                             | 12.706               |
| 2                             | 4.303                |
| 3                             | 3.182                |
| 4                             | 2.776                |
| 5                             | 2.571                |
| 6                             | 2.447                |
| 7                             | 2.365                |
| 8                             | 2.306                |
| 9                             | 2.262                |
| 10                            | 2.228                |
| 12                            | 2.179                |
| 14                            | 2.145                |
| 16                            | 2.120                |
| 18                            | 2.101                |
| 20                            | 2.086                |
| 30                            | 2.042                |
| ∞                             | 1.960                |



### Пример расчёта

Пусть:
- $ b_1 = 15.0 $
- $ S^2_{\text{воспр}} = 4.7 $
- $ N = 8 $

Тогда:
$$
\text{SE}(b_1) = \sqrt{\frac{4.7}{8}} = \sqrt{0.5875} \approx 0.766
$$
$$
t_1 = \frac{|15.0|}{0.766} \approx 19.58
$$

Сравниваем:  
$ 19.58 > 2.120 $ → **коэффициент $ b_1 $ значим**

Это означает: **содержание углерода действительно влияет на предел текучести**.

In [8]:
SE_b = np.sqrt(S2_vospr / n)  # ошибка коэффициента
t_critical = 2.120  # t_{0.025}(16)
t_values = np.abs(b) / SE_b

print(f"Стандартная ошибка: {SE_b:.3f}")
print(f"Табличное значение критерия Стьюдента t: {t_critical}")
print()
print("Коэффициент      | t_набл    | Значимость")
print("-----------------|-----------|-----------")
for j, name in enumerate(['b0', 'b1 (C)', 'b2 (h)', 'b3 (θ)']):
    t_j = t_values[j]
    sig = "Да" if t_j > t_critical else "Нет"
    print(f"{name:<16} | {t_j:9.3f} | {sig:<9}")

Стандартная ошибка: 0.600
Табличное значение критерия Стьюдента t: 2.12

Коэффициент      | t_набл    | Значимость
-----------------|-----------|-----------
b0               |   349.229 | Да       
b1 (C)           |    52.716 | Да       
b2 (h)           |    13.156 | Да       
b3 (θ)           |     5.567 | Да       


## 7. 📌 Проверка адекватности модели по критерию Фишера

После построения модели важно ответить на вопрос:  **Насколько хорошо наша линейная модель описывает реальные данные?**

Для этого используется **критерий Фишера (F-критерий)** — он проверяет **адекватность модели**.


### Как проводится проверка?

#### 1. **Остаточная сумма квадратов (ошибка модели)**

$$
SS_{\text{ост}} = \sum_{i=1}^n (y_i - \hat{y}_i)^2
$$

- $ y_i $ — среднее значение отклика в i-м опыте
- $ \hat{y}_i $ — предсказание модели
- Чем меньше $ SS_{\text{ост}} $, тем лучше модель описывает данные

---

#### 2. **Число степеней свободы остатков**

$$
f_{\text{ост}} = n - p
$$

Где:
- $ n = 8 $ — число опытов
- $ p = 4 $ — число коэффициентов модели ($ b_0, b_1, b_2, b_3 $)
- $ f_{\text{ост}} = 8 - 4 = 4 $



#### 3. **Дисперсия адекватности**

$$
S^2_{\text{ад}} = \frac{SS_{\text{ост}}}{f_{\text{ост}}}
$$

— Это **дисперсия, объясняемая моделью**, но не учтённая ею (т.е. ошибка модели).



#### 4. Расчетный **F-критерий**

$$
F = \frac{S^2_{\text{ад}}}{S^2_{\text{воспр}}}
$$

Сравниваем **ошибку модели** с **ошибкой эксперимента**.

- Если $ F \approx 1 $ → модель хорошая
- Если $ F \gg 1 $ → модель плохая (не хватает членов)

---

#### 5. Табличный ** F-критерий**

Сравниваем $ F $ с табличным значением $ F_{\text{крит}}(f_1, f_2) $, где:
- $ f_1 = f_{\text{ост}} = 4 $ — степени свободы числителя
- $ f_2 = m - 1 = 2 $ — степени свободы знаменателя (из дисперсии воспроизводимости)
- $ \alpha = 0.05 $ — уровень значимости

Если $ F < F_{\text{крит}} $ → модель **адекватна**  
Если $ F \geq F_{\text{крит}} $ → модель **неадекватна**

---

### 📄 Таблица: Критические значения F-критерия при $ \alpha = 0.05 $

| $ f_1 $ ↓ \\ $ f_2 $ → | 1       | 2       | 3       | 4       | 5       | 6       | 10      | ∞       |
|--------------------------|---------|---------|---------|---------|---------|---------|---------|---------|
| **1**                    | 161.4   | 199.5   | 215.7   | 224.6   | 230.2   | 234.0   | 241.9   | 254.3   |
| **2**                    | 18.51   | 19.00   | 19.16   | 19.25   | 19.30   | 19.33   | 19.40   | 19.50   |
| **3**                    | 10.13   | 9.55    | 9.28    | 9.12    | 9.01    | 8.94    | 8.79    | 8.53    |
| **4**                    | 7.71    | 6.94    | 6.59    | 6.39    | 6.26    | 6.16    | 5.96    | 5.63    |
| **5**                    | 6.61    | 5.79    | 5.41    | 5.19    | 5.05    | 4.95    | 4.74    | 4.41    |
| **6**                    | 5.99    | 5.14    | 4.76    | 4.53    | 4.39    | 4.28    | 4.06    | 3.71    |
| **7**                    | 5.59    | 4.74    | 4.35    | 4.12    | 3.97    | 3.87    | 3.64    | 3.28    |
| **8**                    | 5.32    | 4.46    | 4.07    | 3.84    | 3.69    | 3.58    | 3.35    | 2.98    |
| **9**                    | 5.12    | 4.26    | 3.86    | 3.63    | 3.48    | 3.37    | 3.14    | 2.76    |
| **10**                   | 4.96    | 4.10    | 3.71    | 3.48    | 3.33    | 3.22    | 2.98    | 2.60    |

> 🔹 В нашем случае:  
> $ f_1 = 4 $, $ f_2 = 2 $ → $ F_{\text{крит}} = 6.94 $  


### Пример интерпретации

Пусть:
- $ S^2_{\text{ад}} = 30 $
- $ S^2_{\text{воспр}} = 4.7 $
- $ F = 30 / 4.7 \approx 6.38 $

Сравниваем:
- $ 6.38 < 6.94 $ → модель **адекватна**

In [9]:
y_pred = X @ b
residuals = y - y_pred
SS_ost = np.sum(residuals**2)
f_ost = n - len(b)  # 8 - 4 = 4
S2_adeq = SS_ost / f_ost

F_observed = S2_adeq / S2_vospr
F_critical = 6.94  # F(4, 2) при α=0.05
for i in range (0, len(y)):
    print (f'Опыт № {i}: Экспериментальное хначение: {y[i]:2f}, Расчетное значение {y_pred[i]:2f}')

print ('-------------------------')
print(f"Остаточная сумма квадратов: {SS_ost:.2f}")
print(f"Дисперсия адекватности: {S2_adeq:.3f}")
print(f"Расчетный критерий Фишера: {F_observed:.3f}")
print(f"Табличный критерий Фишера (f1=4, f2=2): {F_critical}")

if F_observed < F_critical:
    print("✅ Модель адекватна.")
else:
    print("❌ Модель неадекватна. Рассмотрите взаимодействия.")

Опыт № 0: Экспериментальное хначение: 238.233333, Расчетное значение 229.750000
Опыт № 1: Экспериментальное хначение: 160.100000, Расчетное значение 166.541667
Опыт № 2: Экспериментальное хначение: 240.366667, Расчетное значение 245.525000
Опыт № 3: Экспериментальное хначение: 185.433333, Расчетное значение 182.316667
Опыт № 4: Экспериментальное хначение: 242.833333, Расчетное значение 236.425000
Опыт № 5: Экспериментальное хначение: 164.766667, Расчетное значение 173.216667
Опыт № 6: Экспериментальное хначение: 242.466667, Расчетное значение 252.200000
Опыт № 7: Экспериментальное хначение: 200.766667, Расчетное значение 188.991667
-------------------------
Остаточная сумма квадратов: 495.64
Дисперсия адекватности: 123.910
Расчетный критерий Фишера: 43.093
Табличный критерий Фишера (f1=4, f2=2): 6.94
❌ Модель неадекватна. Рассмотрите взаимодействия.


## 8. 📌  Итоговая модель

In [10]:
# Выводим модель
print(f"ŷ = {b[0]:.1f} {b[1]:+.1f} x₁ {b[2]:+.1f} x₂ {b[3]:+.1f} x₃")

ŷ = 209.4 +31.6 x₁ +7.9 x₂ +3.3 x₃


Где:
- $ x_1 = \frac{C - 0.08}{0.02},\quad $
- $ x_2 = \frac{h - 0.95}{0.25},\quad $
- $x_3 = \frac{\theta - 45}{45} $

Модель:
- построена на однородных данных (проверка по критерию Кохрена ✅)
- коэффициенты $ b_1 $ и $ b_3 $ — значимы (проверка по критерию Стьюдента ✅)
- модель неадекватна (проверка по критерию Фишер ❌)